# matplotlib.pyplot

## Overview

### Introduction: matplotlib and pyplot

Matplotlib[1] is a popular python package for data visualisation. Developed since 2003 [R] as a way of making plots in python using a matlab like syntax. It is currently maintained and actively developed by a large community of open source developers.

Matplotlib features an object oriented interface at the base of which is the `Figure` class. A `Figure` can contain any number of subplots which are reified as objects of the `Axes` class, which contains many of the methods and fields that define the plot. 

Matplotlib.pyplot is a procedural wrapper around Matplotlib's object-oriented interface. The Matplotlib documentation recommends that, with the exception of a few functions that simplify intitialisation and saving of figures (`pyplot.figure`, `pyplot.subplot`, `pyplot.subplots`, and `pyplot.savefig`), the object oriented interface be used when programming, and the stateful pyplot interface be reserved for interactive work [R].

### Importing matplotlib.pyplot

The canonical alias for `matplotlib.pyplot` is `plt` and it is therefore imported by convention as follows:


In [5]:
# import matplotlib.pyplot and assign alias 'plt'
import matplotlib.pyplot as plt

### Quick plots using the procedural interface

### Using the OO interface for greater flexibility  

## matplotlib.pyplot example plots


In [3]:
plt.Axes.

Object `plt.Axes.AxesSubplot` not found.


### 1. Scatterplot with histograms

### 2. Scatterplot matrix

### 3. Bubble plot using geographic data

***

## References

[1] Matplotlib Development Team. (2021). Matplotlib: Visualization with Python. [online] Available from: <https://matplotlib.org/> Accessed: 28th September, 2021

[3] Matplotlib module docstring